In [21]:
import os
import os.path as osp
import json
import pickle
import collections as C
import functools as F
import itertools as I
import random

import math_verify as M
from tqdm import tqdm

In [3]:
'/home/ma-user/local_cache/zhaoxlpku/PromptCoT-DS-Dataset/train.jsonl'
'/home/ma-user/local_cache/zhaoxlpku/PromptCoT-QwQ-Dataset/train.jsonl'
'/home/ma-user/local_cache/dyyyyyyyy/ScaleQuest-Math/train.json'

'/cache/data/MUSTARDSauce'

'/cache/data/MUSTARDSauce'

### PromptCoT-DS-Dataset

In [36]:
load_path = '/home/ma-user/local_cache/zhaoxlpku/PromptCoT-DS-Dataset/train.jsonl'
with open(load_path, 'r') as f:
    data = [json.loads(l) for l in f.readlines()]
print(len(data))

405573


In [37]:
C.Counter(d['source'] for d in data)

Counter({'Numina': 199036,
         'PromptCot': 131723,
         'AQuA': 59294,
         'GSM8K': 6979,
         'AoPS': 4750,
         'MATH': 3791})

In [38]:
data_main = [d for d in data if d['source'] == 'PromptCot']
print(len(data_main))

131723


In [39]:
informal_problem = data_main[0]['prompt'].split('<｜User｜>', 1)[-1]
assert informal_problem.endswith('<｜Assistant｜>')
informal_problem = informal_problem[:-len('<｜Assistant｜>')]
print(informal_problem)

Let $P$ be a point on a regular $n$-gon. A marker is placed on the vertex $P$ and a random process is used to select a vertex $Q$ uniformly from the remaining $n-1$ vertices. The marker is then moved from $P$ to $Q$ and from $Q$ to a vertex $R$ selected uniformly at random from the remaining $n-1$ vertices. This process is repeated $m$ times, and let $A_m$ be the angle determined by the marker at $P$ after $m$ steps. Let $k$ be a positive integer less than $n/2$ such that $A_k$ is in the angle region $\Delta n$ determined by $PQ$ and $PR$ and let $\lfloor x \rfloor$ denote the greatest integer less than or equal to $x$. Find the smallest positive integer $m$ such that $m \leq \lfloor 2n \lfloor \theta \rfloor \rfloor$, where $\theta = \frac{k\pi}{n}$.


In [40]:
informal_solution = data_main[0]['completion'].split('</think>')[-1]
informal_answer = M.parse(informal_solution)[-1]
print(informal_answer)

\lfloor 2n \lfloor \frac{k\pi}{n} \rfloor \rfloor


In [41]:
data_main_processed = []
for i, d in enumerate(data):
    if d['source'] == 'PromptCot':
        informal_problem = data_main[0]['prompt'].split('<｜User｜>', 1)[-1]
        assert informal_problem.endswith('<｜Assistant｜>')
        informal_problem = informal_problem[:-len('<｜Assistant｜>')]
        # print(informal_problem)
        informal_solution = data_main[0]['completion'].split('</think>')[-1]
        informal_answer = M.parse(informal_solution)
        assert len(informal_answer) == 2
        informal_answer = informal_answer[-1]
        # print(informal_answer)
        data_main_processed.append({
            'informal_problem': informal_problem,
            'informal_solution': informal_solution,
            'informal_answer': informal_answer,
            'source_path': load_path,
            'source_idx': i
        })
print(len(data_main_processed))

131723


In [ ]:
with open('/cache/data/fpg_informal_baselines/PromptCoT-DS.processed.jsonl', 'w') as f:
    for d in data_main_processed:
        f.write(json.dumps(d)+'\n')

### PromptCoT-QwQ-Dataset

In [43]:
load_path = '/home/ma-user/local_cache/zhaoxlpku/PromptCoT-QwQ-Dataset/train.jsonl'
with open(load_path, 'r') as f:
    data = [json.loads(l) for l in f.readlines()]
print(len(data))

42887


In [44]:
C.Counter(d['source'] for d in data)

Counter({'PromptCot': 25238, 'Numina': 15421, 'AoPS': 2119, 'MATH': 109})

In [45]:
data_main_processed = []
for i, d in enumerate(data):
    if d['source'] == 'PromptCot':
        informal_problem = data_main[0]['prompt'].split('<｜User｜>', 1)[-1]
        assert informal_problem.endswith('<｜Assistant｜>')
        informal_problem = informal_problem[:-len('<｜Assistant｜>')]
        # print(informal_problem)
        informal_solution = data_main[0]['completion'].split('</think>')[-1]
        informal_answer = M.parse(informal_solution)
        assert len(informal_answer) == 2
        informal_answer = informal_answer[-1]
        # print(informal_answer)
        data_main_processed.append({
            'informal_problem': informal_problem,
            'informal_solution': informal_solution,
            'informal_answer': informal_answer,
            'source_path': load_path,
            'source_idx': i
        })
print(len(data_main_processed))

25238


In [ ]:
with open('/cache/data/fpg_informal_baselines/PromptCoT-QwQ.processed.jsonl', 'w') as f:
    for d in data_main_processed:
        f.write(json.dumps(d)+'\n')

### ScaleQuest-Math

In [79]:
load_path = '/home/ma-user/local_cache/dyyyyyyyy/ScaleQuest-Math/train.json'
with open(load_path, 'r') as f:
    data = [json.loads(l) for l in f.readlines()]
print(len(data))

1003467


In [80]:
data[0]

{'query': 'At what point do the graphs of $y=5x-26$ and $y=-\\frac{3}{4}x+19$ intersect?',
 'response': "To find the point of intersection of the graphs of the equations \\( y = 5x - 26 \\) and \\( y = -\\frac{3}{4}x + 19 \\), we need to find the values of \\( x \\) and \\( y \\) that satisfy both equations simultaneously. This means we need to set the right-hand sides of the equations equal to each other and solve for \\( x \\).\n\nStarting with the equations:\n\\[ y = 5x - 26 \\]\n\\[ y = -\\frac{3}{4}x + 19 \\]\n\nWe set the right-hand sides equal to each other:\n\\[ 5x - 26 = -\\frac{3}{4}x + 19 \\]\n\nTo eliminate the fraction, we can multiply every term by 4:\n\\[ 4(5x - 26) = 4\\left(-\\frac{3}{4}x + 19\\right) \\]\n\\[ 20x - 104 = -3x + 76 \\]\n\nNext, we want to get all the \\( x \\)-terms on one side of the equation and the constant terms on the other side. We do this by adding \\( 3x \\) to both sides:\n\\[ 20x + 3x - 104 = 76 \\]\n\\[ 23x - 104 = 76 \\]\n\nThen, we add 104 

In [81]:
print(data[0]['query'])

At what point do the graphs of $y=5x-26$ and $y=-\frac{3}{4}x+19$ intersect?


In [82]:
print(data[0]['response'])

To find the point of intersection of the graphs of the equations \( y = 5x - 26 \) and \( y = -\frac{3}{4}x + 19 \), we need to find the values of \( x \) and \( y \) that satisfy both equations simultaneously. This means we need to set the right-hand sides of the equations equal to each other and solve for \( x \).

Starting with the equations:
\[ y = 5x - 26 \]
\[ y = -\frac{3}{4}x + 19 \]

We set the right-hand sides equal to each other:
\[ 5x - 26 = -\frac{3}{4}x + 19 \]

To eliminate the fraction, we can multiply every term by 4:
\[ 4(5x - 26) = 4\left(-\frac{3}{4}x + 19\right) \]
\[ 20x - 104 = -3x + 76 \]

Next, we want to get all the \( x \)-terms on one side of the equation and the constant terms on the other side. We do this by adding \( 3x \) to both sides:
\[ 20x + 3x - 104 = 76 \]
\[ 23x - 104 = 76 \]

Then, we add 104 to both sides to isolate the term with \( x \):
\[ 23x = 180 \]

Now, we solve for \( x \) by dividing both sides by 23:
\[ x = \frac{180}{23} \]

With \( x

In [ ]:
data_main_processed = []
for i, d in enumerate(tqdm(data)):
    informal_answer = M.parse(d['response'])
    # assert len(informal_answer) == 2
    informal_answer = informal_answer[-1]
    assert isinstance(informal_answer, str)
    data_main_processed.append({
        'informal_problem': d['query'],
        'informal_solution': d['response'],
        'informal_answer': informal_answer,
        'source_path': load_path,
        'source_idx': i
    })
print(len(data_main_processed))

In [ ]:
# Use `/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/scalequest_processing.py` to process/
raise

In [ ]:
# def worker(args):
#     i, d, load_path = args
#     informal_answer = M.parse(d['response'])
#     # assert len(informal_answer) == 2
#     informal_answer = informal_answer[-1]
#     assert isinstance(informal_answer, str)
#     data_main_processed.append({
#         'informal_problem': d['query'],
#         'informal_solution': d['response'],
#         'informal_answer': informal_answer,
#         'source_path': load_path,
#         'source_idx': i
#     })
#     return data_main_processed

In [1]:
# import multiprocessing
# args_list = [(i, d, load_path) for i, d in enumerate(data)]

# with multiprocessing.Pool(processes=16) as pool:
#     data_main_processed = list(tqdm(
#         pool.imap(worker, args_list),
#         total=len(data),
#         desc="Processing data"
#     ))

In [37]:
with open('/cache/data/fpg_informal_baselines/ScaleQuest-Math.processed.jsonl', 'r') as f:
    data_main_processed = [json.loads(l) for l in f.readlines()]

In [39]:
C.Counter('exception' in d for d in data_main_processed), C.Counter('informal_answer' in d for d in data_main_processed)

(Counter({False: 1003466, True: 1}), Counter({True: 1003466, False: 1}))

In [40]:
for d in data_main_processed:
    if 'exception' in d:
        raise

RuntimeError: No active exception to reraise

In [41]:
print(d['exception'])

IndexError('list index out of range')


In [46]:
d['informal_answer'] = r'\binom{2013^{2012} - 1}{2012}'
d.pop('exception')

"IndexError('list index out of range')"

In [47]:
C.Counter('exception' in d for d in data_main_processed), C.Counter('informal_answer' in d for d in data_main_processed)

(Counter({False: 1003467}), Counter({True: 1003467}))

In [48]:
with open('/cache/data/fpg_informal_baselines/ScaleQuest-Math.processed.jsonl', 'w') as f:
    for d in data_main_processed:
        f.write(json.dumps(d)+'\n')

### MUSTARDSauce

In [50]:
load_path = '/cache/data/MUSTARDSauce/data/subset_filtered'
data = []
for p in os.listdir(load_path):
    with open(osp.join(load_path, p), 'r') as f:
        data.append(json.load(f))
print(len(data))

5866


In [51]:
data[101]

{'problem_name': 'correct_by_msg__HIGH_theorem_proving_Calculus_1_542_round1',
 'informal_statement': 'Consider a random variable X that follows a probability distribution where X can take the values 1, 2, 3, and 4 with the probabilities 1/10, 2/10, 3/10, and 4/10 respectively. Calculate the expected value (mean) of the distribution.',
 'informal_proof': 'The expected value (mean) of a probability distribution is calculated by multiplying each outcome by its probability and then summing these values. \n\nIn our case, the expected value E(X) would be calculated as follows:\n\nE(X) = (1 * 1/10) + (2 * 2/10) + (3 * 3/10) + (4 * 4/10) \n     = 0.1 + 0.4 + 0.9 + 1.6 \n     = 3.0\n\nSo, the expected value of the distribution is 3.0.',
 'formal_proof': 'import data.real.basic\n\n-- make the definition noncomputable as mentioned in the error\nnoncomputable def X : (ℕ → ℝ) := λ x, if x = 1 then 1/10 else if x = 2 then 2/10 else if x = 3 then 3/10 else if x = 4 then 4/10 else 0\n\n-- proof of th

In [52]:
C.Counter(
    'In the following, you are given a' in d['informal_statement'] for d in data
)

Counter({False: 4703, True: 1163})

In [53]:
prompt_str = "In the following, you are given a ``Problem'', a pair of corresponding ``Informal proof'' and ``Formal proof in Lean 3'', along with error messages from a Lean prover corresponding to the ``Formal proof in Lean 3''. Now please carefully modify the ``Formal proof in Lean 3'' section so that it passes the Lean prover without error. You should write the modi fied complete proof in your response."
data_main_processed = []

for d in data:
    if 'In the following' in d['informal_statement']:
        assert d['informal_statement'].startswith(prompt_str)
        data_main_processed.append({
            'informal_statement': d['informal_proof'],
            'formal_proof': d['formal_proof'],
            'source_path': load_path,
            'source_idx': d['problem_name']
        })
    else:
        data_main_processed.append({
            'informal_statement': d['informal_statement'],
            'formal_proof': d['formal_proof'],
            'source_path': load_path,
            'source_idx': d['problem_name']
        })
print(len(data_main_processed))

5866


In [54]:
with open('/cache/data/fpg_informal_baselines/MUSTARDSauce.processed.jsonl', 'w') as f:
    for d in data_main_processed:
        f.write(json.dumps(d)+'\n')

In [92]:
d = random.choice(data_main_processed)
print(d['formal_proof'])

import data.real.basic

/-- The function f(x) = 3x^2 + 2x - 1 -/
def f (x : ℝ) := 3*x^2 + 2*x - 1

/-- The derivative of f -/
def f' (x : ℝ) := 6*x + 2

lemma f_derivative (x : ℝ) : f' x = 6*x + 2 :=
begin
  -- This is a definition, so it is immediate
  refl,
end
